In [7]:
import sys 
#sys.path.append("codes/modules") # add custom Vibe 's modules
sys.path.append("../..") # add standard 's modules
sys.path.append('../modules')

import pyfesom as pf
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap
import numpy as np
from netCDF4 import Dataset
import os
import time

In [8]:
# ==============================================================================
# Running this file loads tracers from old FESOM-REcoM2 output file (oce.mean.nc)
# and saves each tracer in an individual file
# 
#  Input:
#  - mesh_id: Name of mesh, will be added to the netcdf name
#  - meshpath: Speciefies where the target mesh is stored
#  - save_netcdf: If true, netcdf will be created
#  - delete_old_netcdf: If a netcdf file with the same name exists, a new cannot 
#    be made. If set to true, an old netcdf with the same name will be deleted
#
#  Output:
#  - netcdf file for each tracer in the old file
#  
#  During running, keep an eye on the output in the terminal, to see if it 
#  makes sense. 
#
# ==============================================================================

In [9]:
# Loading mesh for run

mesh_id    = 'meshArc4.5'
meshpath   = '/home/hbkoziel/pyfesom/data/mesh/'+mesh_id+'/'            # Defining path where mesh is stored
mesh = pf.load_mesh(meshpath, usepickle=True, get3d=True)                                    # Loading mesh, stores it in mesh.****  
#mesh = pf.fesom_mesh(meshpath, get3d=True)
#mesh.zlevs = -mesh.zlevs                                            # Depth is made negative

tracername = 'area'
first_year = 1985
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
runid	= 'Arc12'

/home/hbkoziel/pyfesom/data/mesh/meshArc4.5/pickle_mesh
2
The usepickle == True)
The pickle file for python 2 exists.
The mesh will be loaded from /home/hbkoziel/pyfesom/data/mesh/meshArc4.5/pickle_mesh


In [10]:
# ==============================================================================
# Settings for netcdf file

save_netcdf       = True                                            # Saves the interpolated field in netcdf file
delete_old_netcdf = True                                            # If a netcdf file with the same name exists it will be deleted
input_directory  = '/scratch/usr/hbkvsk12/hlrn3_work2/results/' # Where the netcdf is saved
output_directory  = '/scratch/usr/hbkoziel/'+runid+'/netcdf/'
plot_netcdf       = True                                           # Reads DIN from the created netcdf file, else it plots the interpolated field (should be the same)

In [12]:
for ind in range(0,len(years)):
    netcdf_name       = 'icecon.'+str(years[ind])+'.monthly.nc'

    # ==============================================================================
    # Loading data

    ncfile	= input_directory+runid+'/Oldfiles/'+runid+'.'+str(years[ind])+'.ice.mean.nc'
    f	= Dataset(ncfile,'r')
    tracer	= f.variables[tracername][:]
    if years[ind] >= 2000:
        t1 = np.mean(tracer[0:15,:],axis=0)
        t2 = np.mean(tracer[15:29,:],axis=0)
        t3 = np.mean(tracer[29:45,:],axis=0)
        t4 = np.mean(tracer[45:60,:],axis=0)
        t5 = np.mean(tracer[60:75,:],axis=0)
        t6 = np.mean(tracer[75:90,:],axis=0)
        t7 = np.mean(tracer[90:106,:],axis=0)
        t8 = np.mean(tracer[106:121,:],axis=0)
        t9 = np.mean(tracer[121:136,:],axis=0)
        t10 = np.mean(tracer[136:152,:],axis=0)
        t11 = np.mean(tracer[152:167,:],axis=0)
        t12 = np.mean(tracer[166:,:],axis=0)
        tracer = [t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12]
    else:
        t1 = tracer[0,:]
        t2 = tracer[1,:]
        t3 = tracer[2,:]
        t4 = tracer[3,:]
        t5 = tracer[4,:]
        t6 = tracer[5,:]
        t7 = tracer[6,:]
        t8 = tracer[7,:]
        t9 = tracer[8,:]
        t10 = tracer[9,:]
        t11 = tracer[10,:]
        t12 = tracer[11,:]
        tracer = [t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12]

    tracershape = np.shape(tracer)

    # ==============================================================================
    # Testing if a netcdf file with the same name exists, if yes, it must be removed
    # to save a new one.

    if os.path.isfile(output_directory+netcdf_name) and delete_old_netcdf:
      os.remove(output_directory+netcdf_name)
      print "The netcdf file "+netcdf_name+" has been deleted to make room for your file of the same name."
    elif os.path.isfile(netcdf_name):
      statement = "The netcdf file "+netcdf_name+" already exists! It must be removed for a new one to be created. This can be done by changing your settings."
      sys.exit(statement)

    if not os.path.isdir(output_directory):
      os.makedirs(output_directory)
      print 'Directory '+output_directory+' has been created'

    # ==============================================================================
    # Creating netcdf file
    if save_netcdf:  
      import time
      w_nc_fid = Dataset(output_directory+netcdf_name, 'w', format='NETCDF4_CLASSIC')      # Create and open new netcdf file to write to
      w_nc_fid.description = f.variables[tracername].description 
      w_nc_fid.history     = 'Created ' + time.ctime(time.time())

      nod3d    = w_nc_fid.createDimension('nod2d', mesh.n2d)               # Create dimension: number of 3d nodes
      time	   = w_nc_fid.createDimension('time', tracershape[0]) 

      w_nc_var = w_nc_fid.createVariable('icecon', 'f8',('time','nod2d'))           # 'DIN' is name of saved variable
                                                                           # 'f8' sets presicion to 64-bit floating point
      w_nc_var.setncatts({'long_name': u'sea ice concentration',\
                          'units': u'0 to 1'})
      w_nc_fid.variables['icecon'][:] = tracer   
      w_nc_fid.close()                                                     # close the new file                

      cwd = os.getcwd()
      print "New netcdf file (",netcdf_name,") has been created."
      print "Location: "+output_directory
    else:
      print 'You have specified not to save your field in netcdf file'

The netcdf file icecon.1985.monthly.nc has been deleted to make room for your file of the same name.
New netcdf file ( icecon.1985.monthly.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
The netcdf file icecon.1986.monthly.nc has been deleted to make room for your file of the same name.
New netcdf file ( icecon.1986.monthly.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
The netcdf file icecon.1987.monthly.nc has been deleted to make room for your file of the same name.
New netcdf file ( icecon.1987.monthly.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
The netcdf file icecon.1988.monthly.nc has been deleted to make room for your file of the same name.
New netcdf file ( icecon.1988.monthly.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
The netcdf file icecon.1989.monthly.nc has been deleted to make room for your file of the same name.
New netcdf file ( icecon.1989.monthly.nc ) has been created.
Locati

In [ ]:
f.variables[tracername].description

In [ ]:
# ==============================================================================
# Plot the surface field

print '***' 
ind=19
if plot_netcdf:
    ncfile	= input_directory+runid+'/Oldfiles/'+runid+'.'+str(years[ind])+'.ice.mean.nc'
    f	= Dataset(ncfile,'r')
    #data2	= f.variables['area'][:]
    data2, elem_no_nan = pf.get_data(f.variables['area'][7,:],mesh,0)

data2=np.copy(data2)
print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
print 'Mean of surface: ',np.mean(data2)
print 'Max and min: ',np.max(data2),np.min(data2)

contours = np.arange(0, 1, .1)

fig = plt.figure(num=1, figsize=(12, 8), facecolor='w', edgecolor='k')
ax=plt.subplot(111)

m = Basemap(projection='robin',lon_0=0) 
x, y = m(mesh.x2, mesh.y2)
m.drawmapboundary(fill_color='0.9')
m.drawcoastlines()
mlabels=[False,False,False,False]
plabels=[True,True,True,True]		    
m.drawparallels(np.arange(-90,90,30),labels=plabels) 
m.drawmeridians(np.arange(m.lonmin,m.lonmax+30,60),labels=mlabels)

#eps=(contours.max()-contours.min())/100.
#data2[data2<=contours.min()]=contours.min()+eps
#data2[data2>=contours.max()]=contours.max()-eps

im=plt.tricontourf(x, y, elem_no_nan, data2, levels=contours, cmap=cm.Spectral_r, extend='both')
label = 'Sea Ice Con'
plt.title(years[ind])
cbar=m.colorbar(im,"bottom", size="5%", pad="2%")
cbar.set_label(label)

#plt.savefig(saving_directory+'ChlNano.png', dpi = 200, bbox_inches='tight')
plt.show()

In [ ]:
ncfile	= input_directory+runid+'/Oldfiles/'+runid+'.'+str(years[ind])+'.ice.mean.nc'
f	= Dataset(ncfile,'r')
#data2	= f.variables['area'][:]
data2, elem_no_nan = pf.get_data(f.variables['area'][:,:],mesh,0)
print data2.shape

In [ ]:
plt.figure(figsize=(10,7))
m.drawmapboundary(fill_color='0.9')
m.drawcoastlines()

plt.tripcolor(x, y, elem2, \
              data2, \
              edgecolors='none',\
              lw = 0.01,
             cmap=cm.Spectral_r,
             vmin = 0,
             vmax = 1)
cbar = plt.colorbar(orientation='horizontal', pad=0.03);
cbar.set_label("Ice Con (%)")
#plt.title('Temperature at 100m depth')
plt.tight_layout()